In [11]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model # type: ignore
from utils.milp import codify_network, codify_network_relaxed 
from time import time
import os

In [12]:
def save_results(file_results, columns, nova_linha):
  if os.path.exists(file_results):
    resultados = pd.read_csv(file_results)
  else:
    resultados = pd.DataFrame(columns=columns)
  resultados.loc[len(resultados)] = nova_linha
  resultados.to_csv(file_results, index=False)

In [13]:
def benchmark_milp():
  # definir array de datasets
  datasets = [
    # {"nome":"iris"},
    # {"nome":"breast_cancer"},
    # {"nome":"wine"},
    # {"nome":"glass"}, # falta treinar os modelos
    {"nome":"digits"}, # resultados sem relevancia
    # {"nome":"mnist"}, # rodar individualmente
  ]
  
  # definir array de modelos
  modelos = [
    "model_1layers_20neurons.h5",
    # "model_2layers_20neurons.h5",
    # "model_3layers_20neurons.h5",
  ]
  
  # definir array de deltas a seres utilizados
  deltas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
  # definir array de resultados
  
  # para cada dataset
  for dataset in datasets:
    data = pd.read_csv(f"datasets/{dataset['nome']}/data.csv")
    # para cada modelo do dataset
    for modelo in modelos:
      print(f"datasets/{dataset['nome']}/models/{modelo}")
      
      # carregar modelo em formato h5
      model_h5 = load_model(
        f"datasets/{dataset['nome']}/models/{modelo}", compile=False)
      
      # 1 - gerar modelos MILP 
      method = "fischetti"
      inicio = time()
      model_milp, output_bounds = codify_network(model_h5, data, method, True)
      fim = time()
      tempo_milp = fim - inicio
      
      # 2 - gerar modelos MILP relaxados
      inicio = time()
      model_milp_relaxed, output_bounds_relaxed = codify_network_relaxed(model_h5,model_milp,data,method,True,output_bounds)
      fim = time()
      tempo_milp_relaxed = fim - inicio
      
      # salvar tempos no csv de resultados 
      save_results(
        "resultados/tempo_milp.csv",
        ["dataset","modelo","metodo","tempo_milp","tempo_milp_relaxed"],
        {
        "dataset": dataset["nome"],
        "modelo": modelo,
        "metodo": method,
        "tempo_milp": tempo_milp,
        "tempo_milp_relaxed": tempo_milp_relaxed
      })  
    
    # para cada instancia do dataset
        # para cada amostra
          # calcular a explicacao
          # salvar o resultado
  # calcular desvio padrão do tempo de explicacao para cada metodo
  

In [14]:
from utils.explanations import get_minimal_explanation, get_explanation_relaxed


def benchmark():
  # definir array de datasets
  datasets = [
    # {"nome":"iris", "n_classes": 3},
    # {"nome":"wine", "n_classes": 3},
    # {"nome":"breast_cancer", "n_classes": 2},
    # {"nome":"glass", "n_classes": 5}, # falta treinar os modelos
    # {"nome":"digits", "n_classes":10}, # resultados sem relevancia
    {"nome":"mnist", "n_classes":10}, # rodar individualmente
    
    # {"nome":"fashio mnist(subset)", "n_classes":10}, # rodar individualmente
    # {"nome":"usps", "n_classes":10}, # rodar individualmente
    # {"nome":"nist special database 1", "n_classes":10}, # rodar individualmente
    # {"nome":"cifar-10(subset)", "n_classes":10}, # rodar individualmente
    # {"nome":"quickdraw dataset(subset)", "n_classes":10}, # rodar individualmente
  ]
  
  # definir array de modelos
  modelos = [
    "model_1layers_20neurons.h5",
    "model_2layers_20neurons.h5",
    "model_3layers_20neurons.h5",
  ]
  
  # definir array de deltas a seres utilizados
  deltas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
  # definir array de resultados
  
  # para cada dataset
  for dataset in datasets:
    data = pd.read_csv(f"datasets/{dataset['nome']}/data.csv")
    # para cada modelo do dataset
    for modelo in modelos:
      # print(f"datasets/{dataset['nome']}/models/{modelo}")
      
      # carregar modelo em formato h5
      model_h5 = load_model(
        f"datasets/{dataset['nome']}/models/{modelo}", compile=False)
      
      # 1 - gerar modelos MILP 
      method = "fischetti"
      model_milp, output_bounds = codify_network(model_h5, data, method, True)
      
      # 2 - gerar modelos MILP relaxados
      model_milp_relaxed, output_bounds_relaxed = codify_network_relaxed(model_h5,model_milp,data,method,True,output_bounds)
      # para cada instancia do dataset
      for index, instance in data.iterrows():
        if index >= 100: # type: ignore
          break
        # a predicao
        network_input = data.iloc[index, :-1] # type: ignore
        network_input = tf.reshape(tf.constant(network_input), (1, -1))
        network_output = model_h5.predict(tf.constant(network_input))[0]
        network_output = tf.argmax(network_output)
        
        # explicacao
        inicio = time()
        (explanation, model) = get_minimal_explanation(
          model_milp.clone(),
          network_input,
          network_output,
          n_classes=dataset["n_classes"],
          method=method,
          output_bounds=output_bounds
        )
        
        fim = time()
        tempo_original = fim - inicio
        len_original = len(explanation)
        
        time_milp_relaxed = []
        len_milp_relaxed = []
        
        for delta in deltas:
          inicio = time()
          (explanation_relaxed_local, model_relaxed_local) = get_explanation_relaxed(
            model_milp_relaxed.clone(),
            network_input,
            network_output,
            n_classes=dataset["n_classes"],
            delta=delta,
          )
          fim = time()
          tempo_relaxed = fim - inicio
          time_milp_relaxed.append(tempo_relaxed)
          len_milp_relaxed.append(len(explanation_relaxed_local))
        
        
        save_results(
          f"resultados/explanations_{dataset['nome']}.csv",
          [
            "dataset", 
            "modelo",
            "instance",
            "time_milp",
            "len_milp",
            "time_milp_relaxed",
            "len_milp_relaxed"
            ],
          {
            "dataset": dataset["nome"], 
            "modelo": modelo,
            "instance": index,
            "time_milp": tempo_original,
            "len_milp": len_original,
            "time_milp_relaxed": time_milp_relaxed,
            "len_milp_relaxed": len_milp_relaxed,
        })

In [15]:
benchmark()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
